In [0]:
%run ./00_config

## 1. PDFからテキスト抽出

In [0]:
# 1. 必要なライブラリのインストール
%pip install pdfplumber databricks-langchain


In [0]:
pdf_volume_path = '/Volumes/my_lab/mydocs_rag/raw_docs/卒論_福永敦史.pdf'

# PDFテキスト抽出関数の定義
import pdfplumber

def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

# テキスト抽出と表示
pdf_text = extract_pdf_text(pdf_volume_path)
print(pdf_text[:1000])  # 先頭1000文字を表示

In [0]:
# 5. 文字列チャンク化関数の定義
import math

def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text, chunk_size=1000)

# 6. チャンクごとにIDを付与しSpark DataFrame化
chunk_data = [(i, chunk) for i, chunk in enumerate(chunks)]
chunk_df = spark.createDataFrame(chunk_data, ["chunk_id", "text"])

# 7. 先頭数件を表示
display(chunk_df.limit(5))

In [0]:
# 5. 文字列チャンク化関数の定義
import math

def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text, chunk_size=1000)

# 6. チャンクごとにIDを付与しSpark DataFrame化
chunk_data = [(i, chunk) for i, chunk in enumerate(chunks)]
chunk_df = spark.createDataFrame(chunk_data, ["chunk_id", "text"])

# Deltaテーブルとして保存
chunk_df.write.option("readChangeFeed", "true").mode("overwrite").saveAsTable(bronze_table_path)

# 保存後の確認表示
display(spark.table(bronze_table_path).limit(5))